In [7]:
import lonboard
from lonboard import viz
import pyrosm
from pyrosm import get_data, OSM
from pyrosm.data import sources
import pandas as pd
import geopandas as gpd
import ast
import json
import re

In [8]:
# Help for the engine: https://github.com/geopandas/geopandas/discussions/3015

water = gpd.read_file("../data/drinking_water.osm.pbf", engine="pyogrio", layer="points")
toilets = gpd.read_file("../data/toilets.osm.pbf", engine="pyogrio", layer="points")

In [9]:
print(len(water), "drinking water POIs and", len(toilets), "toilet POIs")

13610 drinking water POIs and 5243 toilet POIs


In [10]:
water = water[["osm_id", "name", "geometry", "other_tags"]]
toilets = toilets[["osm_id", "name", "geometry", "other_tags"]]

In [11]:
viz([toilets, water])

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [12]:
water_min = water[["osm_id", "geometry"]]
# water_min.to_file("../data/water.geojson", driver="GeoJSON")

In [13]:
toilets_min = toilets[["osm_id", "geometry"]]
# toilets_min.to_file("../data/toilets.geojson", driver="GeoJSON")